## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Gamba,GA,-2.6500,10.0000,70.86,86,56,7.02,broken clouds
1,1,Arraial Do Cabo,BR,-22.9661,-42.0278,76.96,78,20,18.41,few clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,38.82,60,20,9.22,few clouds
3,3,Rosetta,EG,31.4044,30.4164,78.13,79,0,8.99,clear sky
4,4,Kotka,FI,60.4664,26.9458,65.46,92,100,16.98,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))

max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]


preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Arraial Do Cabo,BR,-22.9661,-42.0278,76.96,78,20,18.41,few clouds
3,3,Rosetta,EG,31.4044,30.4164,78.13,79,0,8.99,clear sky
7,7,Port-Gentil,GA,-0.7193,8.7815,75.16,83,100,10.36,overcast clouds
10,10,Los Llanos De Aridane,ES,28.6585,-17.9182,82.15,35,0,17.27,clear sky
13,13,Vila Velha,BR,-20.3297,-40.2925,78.75,80,20,10.36,few clouds
14,14,Iquitos,PE,-3.7481,-73.2472,80.64,94,40,4.61,scattered clouds
15,15,Verbita,RO,44.3000,23.1667,80.60,26,0,1.30,clear sky
16,16,Kavaratti,IN,10.5669,72.6420,80.82,78,100,16.60,overcast clouds
17,17,Naliya,IN,23.2667,68.8333,81.77,82,100,13.00,overcast clouds
19,19,Torbay,CA,47.6666,-52.7314,77.77,76,75,18.41,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
np.where(pd.isnull(preferred_cities_df))

(array([], dtype=int64), array([], dtype=int64))

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Arraial Do Cabo,BR,76.96,few clouds,-22.9661,-42.0278,
3,Rosetta,EG,78.13,clear sky,31.4044,30.4164,
7,Port-Gentil,GA,75.16,overcast clouds,-0.7193,8.7815,
10,Los Llanos De Aridane,ES,82.15,clear sky,28.6585,-17.9182,
13,Vila Velha,BR,78.75,few clouds,-20.3297,-40.2925,
14,Iquitos,PE,80.64,scattered clouds,-3.7481,-73.2472,
15,Verbita,RO,80.60,clear sky,44.3000,23.1667,
16,Kavaratti,IN,80.82,overcast clouds,10.5669,72.6420,
17,Naliya,IN,81.77,overcast clouds,23.2667,68.8333,
19,Torbay,CA,77.77,broken clouds,47.6666,-52.7314,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [38]:
# 7. Drop the rows where there is no Hotel Name.

clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] !=""]

In [39]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [40]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

<dl>

<dt>Hotel Name</dt><dd>{Hotel Name}</dd>

<dt>City</dt><dd>{City}</dd>

<dt>Country</dt><dd>{Country}</dd>

<dt>Max Temp</dt><dd>{Max Temp} °F</dd>

<dt>Weather Description</dt><dd>{Current Description}</dd>

</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.



hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))